In [1]:
from __future__ import print_function
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import wfdb
import time
import random
from sklearn.preprocessing import minmax_scale
import sys

In [3]:
# channel_1 = sys.argv[1]
# channel_2 = sys.argv[2]
channel_1 = 'i'
channel_2 = 'ii'
seed_num = 37

print(seed_num, channel_1, channel_2)

37 i ii


In [4]:
with open('ptbdb_data/RECORDS') as fp:  
    lines = fp.readlines()

In [12]:
len(lines), lines[:5]

(549,
 ['patient001/s0010_re\n',
  'patient001/s0014lre\n',
  'patient001/s0016lre\n',
  'patient002/s0015lre\n',
  'patient003/s0017lre\n'])

In [34]:
files_unhealthy, files_healthy, result = [], [], []

for file_ in lines:
    file_path = "ptbdb_data/" + file_[:-1] + ".hea"
    
    # read header to determine class
    if 'Myocardial infarction' in open(file_path).read():
        files_unhealthy.append(file_)
        result.append(0)
        
    if 'Healthy control' in open(file_path).read():
        files_healthy.append(file_)
        result.append(1)


In [17]:
len(files_unhealthy), len(files_healthy)

(368, 80)

In [18]:
files_unhealthy[0]

'patient001/s0010_re\n'

In [19]:
# IDEALLY, SEED SHOULD BE FLAG AND SHOULD SEE TENSORBOARDS FOR 10 SEEDS

In [36]:
# check if there are patients that have both healthy and unhealthy recordings

for idx, file_ in enumerate(lines):
    print(file_[:-1], result[idx])

patient001/s0010_re 0
patient001/s0014lre 0
patient001/s0016lre 0
patient002/s0015lre 0
patient003/s0017lre 0
patient004/s0020are 0
patient004/s0020bre 0
patient005/s0021are 0
patient005/s0021bre 0
patient005/s0025lre 0
patient005/s0031lre 0
patient005/s0101lre 0
patient006/s0022lre 0
patient006/s0027lre 0
patient006/s0064lre 0
patient007/s0026lre 0
patient007/s0029lre 0
patient007/s0038lre 0
patient007/s0078lre 0
patient008/s0028lre 0
patient008/s0037lre 0
patient008/s0068lre 0
patient009/s0035_re 0
patient010/s0036lre 0
patient010/s0042lre 0
patient010/s0061lre 0
patient011/s0039lre 0
patient011/s0044lre 0
patient011/s0049lre 0
patient011/s0067lre 0
patient012/s0043lre 0
patient012/s0050lre 0
patient013/s0045lre 0
patient013/s0051lre 0
patient013/s0072lre 0
patient014/s0046lre 0
patient014/s0056lre 0
patient014/s0071lre 0
patient015/s0047lre 0
patient015/s0057lre 0
patient015/s0152lre 0
patient016/s0052lre 0
patient016/s0060lre 0
patient016/s0076lre 0
patient017/s0053lre 0
patient017

IndexError: list index out of range

In [31]:
# get list of patient_ids, split this, then populate unhealthy/healthy according to this split

# num_patients = 294
patient_ids = np.arange(294) + 1
# NO SINCE THERE ARE GAPS, e.g. 132, 134

# shuffle
np.random.seed(int(seed_num))
np.random.shuffle(patient_ids)

# (healthy / unhealthy) = (80 / 368)
# should do 80% train: (64 / 295) 



# WANT SPLIT BY PATIENT, AND EVEN HEALTHY/UNHEALTHY

In [32]:
patient_ids

array([201, 169,  11,  83, 186,  95, 113, 281, 149, 257, 122, 225,  69,
       199, 215, 131, 228,  28, 255,  56, 283, 273, 103, 258, 246, 137,
       223,  67, 148, 289, 111,   4,   5, 104,  72, 259, 170,   2, 256,
       194,  93, 279, 287, 147, 218, 118,  31, 234, 212, 294,   3, 276,
       135,  90, 288, 244, 172,  98, 238,  48, 293, 173, 175, 132, 125,
       165, 140,  73,  68, 105, 285, 227, 205,  39,  41,   9, 157, 121,
         6, 213, 110,  38, 102, 187,  82, 180,  49,  17, 268, 162, 229,
       161, 151, 263, 237,  33, 224, 198, 195, 210,  75,   7, 166, 262,
        21, 241, 181, 146, 176,  94, 133, 277, 286, 177, 196, 266, 260,
       156, 243, 128, 182,  79, 250, 254,  18, 142, 188, 264, 124, 269,
        84,  97, 232, 290,  13,  77, 231, 109,  24,  71,  99,  15, 191,
       114, 209, 119,  70,  60, 292,  92, 139, 126,  76, 115,  14,  45,
       278,  35,  27, 275, 152,  46, 247, 134, 219, 217, 221, 236,  96,
       220,  87, 183,  25,  89,   1, 270,  19, 144,  86, 192,  1

In [54]:
# split files as usual, then move overlapping patients to train. try a bunch of seeds.

# load real data (ptbdb)
with open('ptbdb_data/RECORDS') as fp:  
    lines = fp.readlines()

files_unhealthy, files_healthy = [], []

for file in lines:
    file_path = "ptbdb_data/" + file[:-1] + ".hea"
    
    # read header to determine class
    if 'Myocardial infarction' in open(file_path).read():
        files_unhealthy.append(file)
        
    if 'Healthy control' in open(file_path).read():
        files_healthy.append(file)

# shuffle data (cross-validation)
np.random.seed(int(seed_num))
np.random.shuffle(files_unhealthy)
np.random.shuffle(files_healthy)

healthy_train = files_healthy[:int(0.8*len(files_healthy))]
healthy_val = files_healthy[int(0.8*len(files_healthy)):]
unhealthy_train = files_unhealthy[:int(0.8*len(files_unhealthy))]
unhealthy_val = files_unhealthy[int(0.8*len(files_unhealthy)):]

In [55]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [56]:
patient_ids_unhealthy_train = [element[:10] for element in unhealthy_train]
patient_ids_unhealthy_val = [element[:10] for element in unhealthy_val]
patient_ids_healthy_train = [element[:10] for element in healthy_train]
patient_ids_healthy_val = [element[:10] for element in healthy_val]

intersection_unhealthy = intersection(patient_ids_unhealthy_train, patient_ids_unhealthy_val)
intersection_healthy = intersection(patient_ids_healthy_train, patient_ids_healthy_val)

print(intersection_unhealthy)
print()
print(intersection_healthy)

['patient083', 'patient082', 'patient080', 'patient085', 'patient084', 'patient100', 'patient089', 'patient050', 'patient223', 'patient158', 'patient078', 'patient079', 'patient076', 'patient077', 'patient072', 'patient073', 'patient014', 'patient033', 'patient030', 'patient031', 'patient011', 'patient038', 'patient039', 'patient018', 'patient047', 'patient098', 'patient099', 'patient017', 'patient051', 'patient096', 'patient097', 'patient091', 'patient197', 'patient093', 'patient292', 'patient293', 'patient045', 'patient008', 'patient042', 'patient041', 'patient040', 'patient065', 'patient007', 'patient005', 'patient048', 'patient021', 'patient020', 'patient023', 'patient022', 'patient016']

['patient198', 'patient180', 'patient174', 'patient236', 'patient233']


In [57]:
# unhealthy
move_to_train = intersection_unhealthy[:int(0.5*len(intersection_unhealthy))]
move_to_val = intersection_unhealthy[int(0.5*len(intersection_unhealthy)):]

for patient_id in move_to_train:
    
    in_val = []
    
    # find and remove all files in val
    for file_ in unhealthy_val:
        if file_[:10] == patient_id:
            in_val.append(file_)
            unhealthy_val.remove(file_)
            
    # add to train
    for file_ in in_val:
        unhealthy_train.append(file_)
       
    
for patient_id in move_to_val:
    
    in_train = []
    
    # find and remove all files in val
    for file_ in unhealthy_train:
        if file_[:10] == patient_id:
            in_train.append(file_)
            unhealthy_train.remove(file_)
            
    # add to train
    for file_ in in_train:
        unhealthy_val.append(file_)
    
    
# healthy
move_to_train = intersection_healthy[:int(0.5*len(intersection_healthy))]
move_to_val = intersection_healthy[int(0.5*len(intersection_healthy)):]

for patient_id in move_to_train:
    
    in_val = []
    
    # find and remove all files in val
    for file_ in healthy_val:
        if file_[:10] == patient_id:
            in_val.append(file_)
            healthy_val.remove(file_)
            
    # add to train
    for file_ in in_val:
        healthy_train.append(file_)
        

for patient_id in move_to_val:
    
    in_train = []
    
    # find and remove all files in val
    for file_ in healthy_train:
        if file_[:10] == patient_id:
            in_train.append(file_)
            healthy_train.remove(file_)
            
    # add to train
    for file_ in in_train:
        healthy_val.append(file_)
    

In [58]:
patient_ids_unhealthy_train = [element[:10] for element in unhealthy_train]
patient_ids_unhealthy_val = [element[:10] for element in unhealthy_val]
patient_ids_healthy_train = [element[:10] for element in healthy_train]
patient_ids_healthy_val = [element[:10] for element in healthy_val]

intersection_unhealthy = intersection(patient_ids_unhealthy_train, patient_ids_unhealthy_val)
intersection_healthy = intersection(patient_ids_healthy_train, patient_ids_healthy_val)

print(intersection_unhealthy)
print()
print(intersection_healthy)

[]

[]


In [59]:
unhealthy_train[0]

'patient087/s0321lre\n'